In [1]:
# General imports
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# TF imports
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Lambda
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Neurophox imports
from neurophox.numpy import *
from neurophox.tensorflow import *
from neurophox.ml.nonlinearities import cnorm, cnormsq
from neurophox.initializers import *
from neurophox.components import *
from neurophox.helpers import *

In [2]:
plt.rcParams['text.usetex'] = True 
plt.rcParams['text.latex.preamble'] = r"\usepackage{siunitx} \usepackage{amsmath} \usepackage{sansmathfonts} \usepackage[T1]{fontenc} \renewcommand*\familydefault{\sfdefault}"
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Import dataset
(X_raw, y), (X_test_raw, y_test) = fashion_mnist.load_data()

In [4]:
X_train = X_raw.reshape(X_raw.shape[0], X_raw.shape[1] * X_raw.shape[2])
X_train = X_train.astype('float32') / 255
X_test = X_test_raw.reshape(X_test_raw.shape[0], X_test_raw.shape[1] * X_test_raw.shape[2])
X_test = X_test.astype('float32') / 255

In [5]:
y = to_categorical(y)
y_test = to_categorical(y_test)

In [6]:
# Show the shapes of the data.
print("Training Images:", X_train.shape)
print("Testing Images:", X_test.shape)
print("Training Labels:", y.shape)
print("Test Labels:", y_test.shape)

Training Images: (60000, 784)
Testing Images: (10000, 784)
Training Labels: (60000, 10)
Test Labels: (10000, 10)


In [7]:
def normalize(X):
    return X / abs(X).mean(axis=0, keepdims=True)

def inputFFT(X_raw, X_test_raw, r):
    '''
        FTs the input images
    '''
    c = X_raw.shape[1] // 2

    # FTing the image data
    X_ft = np.fft.fftshift(np.fft.fft2(X_raw), axes=(1, 2))
    X_test_ft = np.fft.fftshift(np.fft.fft2(X_test_raw), axes=(1, 2))

    # Cropping out the outer pixels
    X_ft = X_ft[:, c-r:c+r, c-r:c+r]
    X_test_ft = X_test_ft[:, c-r:c+r, c-r:c+r]

    # Flatten image
    X_ft = X_ft.reshape(X_ft.shape[0], -1)
    X_test_ft = X_test_ft.reshape(X_test_ft.shape[0], -1)

    # Normalizing
    X_ft = normalize(X_ft).astype(np.complex64)
    X_test_ft = normalize(X_test_ft).astype(np.complex64)
    return X_ft, X_test_ft

X_ft, X_test_ft = inputFFT(X_raw, X_test_raw, 3)

In [8]:
X_ft.shape

(60000, 36)

In [9]:
def generate_network_v2(N, N_classes, theta_init_name='haar_rect', phi_init_name='random_phi'):
    """ 

    Args:
        N (int): Size of the input layer
        N_classes (int, optional): _description_. Defaults to 10.
        L (int, optional): _description_. Defaults to 1.
        theta_init_name (str, optional): _description_. Defaults to 'haar_rect'.
        phi_init_name (str, optional): _description_. Defaults to 'random_phi'.

    Returns:
        Sequential: _description_
    """
    layers=[]
    layers.append(RM(N, theta_init_name=theta_init_name, phi_init_name=phi_init_name))
    # layers.append(Lambda(lambda x: x[:, :50]))
    # layers.append(RM(50, theta_init_name=theta_init_name, phi_init_name=phi_init_name))
    layers.append(Activation(cnormsq))
    layers.append(Lambda(lambda x: tf.math.real(x[:, :N_classes])))
    layers.append(tf.keras.layers.Softmax(axis=-1))

    return Sequential(layers)

In [10]:
epochs = 20
batch_size = 256
N_classes = 10

In [11]:
onn_model = generate_network_v2(36, 10)

In [12]:
onn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
history = onn_model.fit(X_ft, y, epochs=epochs, batch_size=batch_size, validation_data=(X_test_ft, y_test), verbose=2)

Epoch 1/20
235/235 - 34s - loss: 1.8813 - accuracy: 0.4254 - val_loss: 1.3514 - val_accuracy: 0.6080 - 34s/epoch - 144ms/step
Epoch 2/20
235/235 - 15s - loss: 1.1032 - accuracy: 0.6813 - val_loss: 0.9968 - val_accuracy: 0.7104 - 15s/epoch - 63ms/step
Epoch 3/20
235/235 - 15s - loss: 0.9033 - accuracy: 0.7369 - val_loss: 0.8895 - val_accuracy: 0.7377 - 15s/epoch - 62ms/step
Epoch 4/20
235/235 - 15s - loss: 0.8232 - accuracy: 0.7587 - val_loss: 0.8352 - val_accuracy: 0.7569 - 15s/epoch - 66ms/step
Epoch 5/20
235/235 - 16s - loss: 0.7784 - accuracy: 0.7713 - val_loss: 0.8011 - val_accuracy: 0.7642 - 16s/epoch - 68ms/step
Epoch 6/20
235/235 - 16s - loss: 0.7491 - accuracy: 0.7788 - val_loss: 0.7752 - val_accuracy: 0.7731 - 16s/epoch - 70ms/step
Epoch 7/20
235/235 - 16s - loss: 0.7281 - accuracy: 0.7840 - val_loss: 0.7584 - val_accuracy: 0.7797 - 16s/epoch - 68ms/step
Epoch 8/20
235/235 - 15s - loss: 0.7114 - accuracy: 0.7883 - val_loss: 0.7448 - val_accuracy: 0.7794 - 15s/epoch - 63ms/step

In [28]:
def grid_common_mode_flow(external_phases: np.ndarray, gamma: np.ndarray, basis: str = "sm"):
    """In a grid mesh (e.g., triangular, rectangular meshes), arrange phases according to single-mode (:code:`sm`),
       differential mode (:code:`diff`), or max-:math:`\\pi` (:code:`maxpi`, all external phase shifts are at most
       :math:`\\pi`). This is done using a procedure called "common mode flow" where common modes are shifted
       throughout the mesh until phases are correctly set.

    Args:
        external_phases: external phases in the grid mesh
        gamma: input phase shifts
        basis: single-mode (:code:`sm`), differential mode (:code:`diff`), or max-:math:`\\pi` (:code:`maxpi`)

    Returns:
        new external phases shifts and new gamma resulting

    """
    units, num_layers = external_phases.shape
    phase_shifts = np.hstack((gamma[:, np.newaxis], external_phases)).T
    new_phase_shifts = np.zeros_like(external_phases.T)

    for i in range(num_layers):
        current_layer = num_layers - i
        start_idx = (current_layer - 1) % 2
        end_idx = units - (current_layer + units - 1) % 2
        # calculate phase information
        upper_phase = phase_shifts[current_layer][start_idx:end_idx][::2]
        lower_phase = phase_shifts[current_layer][start_idx:end_idx][1::2]
        upper_phase = np.mod(upper_phase, 2 * np.pi)
        lower_phase = np.mod(lower_phase, 2 * np.pi)
        if basis == "sm":
            new_phase_shifts[-i - 1][start_idx:end_idx][::2] = upper_phase - lower_phase
        # assign differential phase to the single mode layer and keep common mode layer
        else:
            phase_diff = upper_phase - lower_phase
            phase_diff[phase_diff > np.pi] -= 2 * np.pi
            phase_diff[phase_diff < -np.pi] += 2 * np.pi
            if basis == "diff":
                new_phase_shifts[-i - 1][start_idx:end_idx][::2] = phase_diff / 2
                new_phase_shifts[-i - 1][start_idx:end_idx][1::2] = -phase_diff / 2
            elif basis == "pimax":
                new_phase_shifts[-i - 1][start_idx:end_idx][::2] = phase_diff * (phase_diff >= 0)
                new_phase_shifts[-i - 1][start_idx:end_idx][1::2] = -phase_diff * (phase_diff < 0)
        # update the previous layer with the common mode calculated for the current layer\
        phase_shifts[current_layer] -= new_phase_shifts[-i - 1]
        phase_shifts[current_layer - 1] += np.mod(phase_shifts[current_layer], 2 * np.pi)
        phase_shifts[current_layer] = 0
    new_gamma = np.mod(phase_shifts[0], 2 * np.pi)
    return np.mod(new_phase_shifts.T, 2 * np.pi), new_gamma

def checkerboard_to_param(checkerboard: np.ndarray, units: int):
    param = np.zeros((units, units // 2))
    if units % 2:
        param[::2, :] = checkerboard.T[::2, :-1:2]
    else:
        param[::2, :] = checkerboard.T[::2, ::2]
    param[1::2, :] = checkerboard.T[1::2, 1::2]
    return param

In [56]:
def clements_decomposition(u: np.ndarray, pbar_handle: Callable = None, smmzi: bool = False) -> RM:
    """Clements decomposition of unitary matrix :math:`U` to output a NumPy rectangular mesh layer

    Args:
        u: unitary matrix :math:`U` to be decomposed into pairwise operators.
        pbar_handle: Useful for larger matrices

    Returns:
        The :code:`RMNumpy` layer that outputs the unitary :math:`U`

    """
    u_hat = u.T.copy()
    n = u.shape[0]
    # odd and even layer dimensions
    theta_checkerboard = np.zeros_like(u, dtype=NP_FLOAT)
    phi_checkerboard = np.zeros_like(u, dtype=NP_FLOAT)
    phi_checkerboard = np.hstack((np.zeros((n, 1)), phi_checkerboard))
    iterator = pbar_handle(range(n - 1)) if pbar_handle else range(n - 1)
    MZI = SMMZI if smmzi else BlochMZI
    for i in iterator:
        if i % 2:
            for j in range(i + 1):
                pairwise_index = n + j - i - 2
                target_row, target_col = n + j - i - 1, j
                theta = np.arctan(np.abs(u_hat[target_row - 1, target_col] / u_hat[target_row, target_col])) * 2
                phi = np.angle(u_hat[target_row, target_col] / u_hat[target_row - 1, target_col])
                mzi = MZI(theta, phi, hadamard=False, dtype=np.complex64)
                left_multiplier = mzi.givens_rotation(units=n, m=pairwise_index)
                u_hat = left_multiplier @ u_hat
                theta_checkerboard[pairwise_index, j] = theta
                phi_checkerboard[pairwise_index, j] = -phi + np.pi
                phi_checkerboard[pairwise_index + 1, j] = np.pi
        else:
            for j in range(i + 1):
                pairwise_index = i - j
                target_row, target_col = n - j - 1, i - j
                theta = np.arctan(np.abs(u_hat[target_row, target_col + 1] / u_hat[target_row, target_col])) * 2
                phi = np.angle(-u_hat[target_row, target_col] / u_hat[target_row, target_col + 1])
                mzi = BlochMZI(theta, phi, hadamard=False, dtype=np.complex64)
                right_multiplier = mzi.givens_rotation(units=n, m=pairwise_index)
                u_hat = u_hat @ right_multiplier.conj().T
                theta_checkerboard[pairwise_index, -j - 1] = theta
                phi_checkerboard[pairwise_index, -j - 1] = phi + np.pi

    diag_phases = np.angle(np.diag(u_hat))
    theta = checkerboard_to_param(np.fliplr(theta_checkerboard), n)
    phi_checkerboard = np.fliplr(phi_checkerboard)
    if n % 2:
        phi_checkerboard[:, :-1] += np.fliplr(np.diag(diag_phases))
    else:
        phi_checkerboard[:, 1:] += np.fliplr(np.diag(diag_phases))
        phi_checkerboard[-1, 2::2] += np.pi / 2  # neurophox layers assume pi / 2 phase shift in even layer "bounces"
        phi_checkerboard[0, 2::2] += np.pi / 2

    gamma = phi_checkerboard[:, 0]
    external_phases = phi_checkerboard[:, 1:]
    phi, gamma = grid_common_mode_flow(external_phases, gamma=gamma)
    phi = checkerboard_to_param(phi, n)

    # for some reason, we need to adjust gamma at the end in this strange way (found via trial and error...):
    gamma_adj = np.zeros_like(gamma)
    gamma_adj[1::4] = 1
    gamma_adj[2::4] = 1
    gamma += np.pi * (1 - gamma_adj) if (n // 2) % 2 else np.pi * gamma_adj
    gamma = np.mod(gamma, 2 * np.pi)
    ans = RM(units=n)
    ans.theta = theta
    ans.phi = phi
    ans.gamma = gamma
    return ans, theta, phi, gamma

In [57]:
onn_model.layers[0].matrix[:18, :18]

array([[ 1.14878928e-02-0.01262338j,  6.70793205e-02+0.05961495j,
         4.77957651e-02-0.07122278j,  8.89131874e-02+0.07028865j,
         7.71195889e-02+0.07472196j, -1.03119455e-01+0.09505212j,
        -6.75403401e-02+0.044178j  , -3.15255225e-02-0.02527959j,
         9.49990228e-02+0.00777542j,  7.20604509e-03-0.08301166j,
        -2.48817533e-01-0.19703491j,  7.35820085e-02+0.03837118j,
         3.83044779e-01+0.16754557j,  1.51269678e-02-0.1424724j ,
         2.58264631e-01+0.03855673j, -1.76287338e-01+0.1585699j ,
        -1.03336945e-02-0.14222813j,  2.04745770e-01-0.19242093j],
       [ 3.82427834e-02+0.13707861j,  7.09580481e-02+0.08091827j,
        -7.95826316e-04-0.11451185j, -4.25149277e-02+0.06210376j,
         2.45771874e-02+0.07510665j,  1.70334056e-01-0.11806981j,
         1.11847911e-02-0.01130337j,  4.80662137e-02-0.14980897j,
        -2.64570117e-05+0.03126169j,  6.15991130e-02+0.01783556j,
        -2.46740486e-02+0.14585331j, -3.60900201e-02+0.17909554j,
        -

In [85]:
bruh.matrix

TypeError: in user code:

    File "c:\ProgramData\Anaconda3\envs\neurophoxgpu\lib\site-packages\neurophox\tensorflow\generic.py", line 491, in transform  *
        mesh_phases, mesh_layers = self.phases_and_layers
    File "c:\ProgramData\Anaconda3\envs\neurophoxgpu\lib\site-packages\neurophox\tensorflow\generic.py", line 537, in phases_and_layers
        mesh_layers = self.mesh.mesh_layers(mesh_phases)
    File "c:\ProgramData\Anaconda3\envs\neurophoxgpu\lib\site-packages\neurophox\tensorflow\generic.py", line 437, in mesh_layers
        s11 = (self.epp * internal_psl - self.enn * roll_tensor(internal_psl, up=True))

    TypeError: Input 'y' of 'Mul' Op has type complex128 that does not match type complex64 of argument 'x'.


array([5.5288092 , 4.39412237, 4.60936246, 5.36511264, 0.23268382,
       5.11674463, 5.29026315, 1.99555937, 0.72461963, 2.77564502,
       4.01157301, 1.96270817, 0.55634496, 0.31580667, 0.62399355,
       1.02821279, 2.10094829, 2.7950532 ])

In [60]:
t.shape

(18, 9)

In [87]:
core11 = generate_network_v2(18, 10)
core11.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
core12 = generate_network_v2(18, 10)
core12.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
core21 = generate_network_v2(18, 10)
core21.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
core22 = generate_network_v2(18, 10)
core22.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
def generate_network_v3(N, N_classes, theta_init_name='haar_rect', phi_init_name='random_phi'):
    """ 

    Args:
        N (int): Size of the input layer
        N_classes (int, optional): _description_. Defaults to 10.
        L (int, optional): _description_. Defaults to 1.
        theta_init_name (str, optional): _description_. Defaults to 'haar_rect'.
        phi_init_name (str, optional): _description_. Defaults to 'random_phi'.

    Returns:
        Sequential: _description_
    """
    layers=[]
    layers.append(RM(N, theta_init_name=theta_init_name, phi_init_name=phi_init_name))
    layers.append(Lambda(lambda x: x[:, :200]))
    layers.append(RM(200, theta_init_name=theta_init_name, phi_init_name=phi_init_name))
    layers.append(Activation(cnormsq))
    layers.append(Lambda(lambda x: tf.math.real(x[:, :N_classes])))
    layers.append(tf.keras.layers.Softmax(axis=-1))

    return Sequential(layers)

In [10]:
onn2 = generate_network_v3(784, 10)

In [11]:
onn2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = onn2.fit(X_train, y, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10


: 

In [52]:
import torch

In [58]:
M, N, K = 15, 20, 18

A = torch.rand((M, K))
B = torch.rand((K, N))

In [59]:
block_M, block_N, block_K = M // 5, N // 5, K // 6

output = torch.zeros((M, N))
total_load = 0
total_write = 0

for index_M in range(0, M, block_M):
    start_M = index_M
    end_M = index_M + block_M

    for index_N in range(0, N, block_N):
        start_N = index_N
        end_N = index_N + block_N
        accumulator = torch.zeros((block_M, block_N))
        for index_K in range(0, K, block_K):
            start_K = index_K
            end_K = index_K + block_K

            tile_A = A[start_M:end_M, start_K:end_K]
            total_load += tile_A.numel()
            tile_B = B[start_K:end_K, start_N:end_N]
            total_load += tile_B.numel()
            # @ means matmul in numpy and pytorch
            accumulator += tile_A @ tile_B
        output[start_M:end_M, start_N:end_N] = accumulator
        total_write += accumulator.numel()

In [60]:
assert torch.allclose(output, A @ B)
print("total load from GM:", total_load)
print("total write to GM:", total_write)

total load from GM: 3150
total write to GM: 300
